# Steps to Do
1. Remove Stop words (spacy tokenize, check stop words, rejoin sentences as string). Lower case, remove punctuation, (remove digits??). 
2. Count Vectorizer & Tfidf Transformer, Rocchio.
3. K-Fold Training Accuracy, 
4. Predict Labels and submit on website.
5. Also try Oversampling (SMOTE) 

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import time
import re
import spacy
import random

In [2]:
nlp_spacy = spacy.load('en_core_web_sm')

In [3]:
english_stop_words = nlp_spacy.Defaults.stop_words
print(len(english_stop_words))

326


In [4]:
"""import nltk
nltk.download('stopwords')"""

"import nltk\nnltk.download('stopwords')"

In [5]:
"""from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
english_stop_words = stopwords.words('english')
print(english_stop_words)"""


"from nltk.tokenize import sent_tokenize, word_tokenize\nfrom nltk.corpus import stopwords\nenglish_stop_words = stopwords.words('english')\nprint(english_stop_words)"

# Reading Datasets

In [6]:
train_df = pd.read_csv("/content/drive/MyDrive/CS 505 Project/Train.csv")
print(train_df.shape)
print(train_df.Label.value_counts())
train_df.head()

(1436, 3)
POLITICS                279
SOCIAL                  152
RELIGION                147
LAW/ORDER               136
SOCIAL ISSUES           134
HEALTH                  127
ECONOMY                  86
FARMING                  78
SPORTS                   49
EDUCATION                43
RELATIONSHIPS            39
WILDLIFE/ENVIRONMENT     36
OPINION/ESSAY            26
LOCALCHIEFS              25
CULTURE                  23
WITCHCRAFT               16
MUSIC                    15
TRANSPORT                11
ARTS AND CRAFTS           7
FLOODING                  7
Name: Label, dtype: int64


,ID,Text,Label
0,ID_AASHwXxg,Mwangonde: Khansala wachinyamata Akamati achi...,POLITICS
1,ID_AGoFySzn,MCP siidakhutire ndi kalembera Chipani cha Ma...,POLITICS
2,ID_AGrrkBGP,Bungwe la MANEPO Lapempha Boma Liganizire Anth...,HEALTH
3,ID_AIJeigeG,Ndale zogawanitsa miyambo zanyanya Si zachile...,POLITICS
4,ID_APMprMbV,Nanga wapolisi ataphofomoka? Masiku ano sichi...,LAW/ORDER


In [7]:
test_df = pd.read_csv("/content/drive/MyDrive/CS 505 Project/Test.csv")
print(test_df.shape)
test_df.head()

(620, 2)


,ID,Text
0,ID_ADHEtjTi,Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1,ID_AHfJktdQ,Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2,ID_AUJIHpZr,Anatcheleza: Akundiopseza a gogo wanga Akundi...
3,ID_AUKYBbIM,Ulova wafika posauzana Adatenga digiri ya uph...
4,ID_AZnsVPEi,"Dzombe kukoma, koma Kuyambira makedzana, pant..."


In [8]:
stop_df = pd.read_csv("/content/drive/MyDrive/CS 505 Project/stopwords.csv")
print(stop_df.shape)
stop_df.head()

(1000, 3)


,Number,Chichewa,in English
0,1,monga,as
1,2,ine,I
2,3,wake,his
3,4,kuti,that
4,5,iye,he


# Pre-processing

In [9]:
chichewa_stop_words = dict(zip(stop_df["Chichewa"], stop_df["in English"]))

In [10]:
random.sample(list(chichewa_stop_words.items()),5)

[('chakudya', 'feed'),
 ('adzauka', 'rise'),
 ('osangalatsa', 'fun'),
 ('kujowina', 'join'),
 ('osiyana', 'separate')]

In [11]:
# Lower Cased, keep only ASCII characters, tokenized, remove punctuation, remove stop words, removed digits

def preprocess(text):
  #print("Original",text)

  text_2 = text.lower()

  # keep only ascii characters
  text_3 = re.sub(r"[^a-zA-ZÀ-ÿ]", " ", text_2)

  #print("ASCII,LOWER-",text_3)

  spacy_object = nlp_spacy(text_3)
  sentences = list(spacy_object.sents) 
  tokens = []
  for sentence in sentences:
    for token in sentence:
      if not(token.is_punct): #and token.isdigit()):
        if token.text in chichewa_stop_words.keys():
          #print("Dictionary used")
          if not (chichewa_stop_words[token.text] in english_stop_words):  #is_stop:
            #tk = token.text.lower()
            #tokens.append(tk)
            tokens.append(token.text)  
          #else:
            #print("Stop Word Removed",token.text,"--",chichewa_stop_words[token.text])
        else:
          #tk = token.text.lower()
          #tokens.append(tk)
          tokens.append(token.text)

  tokenized = " ".join(tokens)
  result = ''.join([i for i in tokenized if not i.isdigit()])
  
  #print("\nFINAL")
  #print(" ".join(tokens))
  #print("\n")
  
  return result

In [12]:
train_df_2 = train_df.copy()
train_df_2.Text = train_df.Text.apply(preprocess)
train_df_2

,ID,Text,Label
0,ID_AASHwXxg,mwangonde khansala wachinyamata akamati ac...,POLITICS
1,ID_AGoFySzn,mcp siidakhutire kalembera chipani cha malaw...,POLITICS
2,ID_AGrrkBGP,bungwe la manepo lapempha boma liganizire anth...,HEALTH
3,ID_AIJeigeG,ndale zogawanitsa miyambo zanyanya si zachil...,POLITICS
4,ID_APMprMbV,nanga wapolisi ataphofomoka masiku ano sic...,LAW/ORDER
...,...,...,...
1431,ID_zmTmmEio,eni minibus ati ali ufulu wokweza mitengo mwa ...,TRANSPORT
1432,ID_znOlIaGQ,kachali apepesa kulankhula motumbwa kuthe ...,POLITICS
1433,ID_znracTjN,mawu supports non fiction writers the malawi...,EDUCATION
1434,ID_ztdsmmva,tame mwawa phwete ndiye kudya kwake sewero...,SOCIAL ISSUES


In [13]:
test_df_2 = test_df.copy()
test_df_2.Text = test_df.Text.apply(preprocess)
test_df_2

,ID,Text
0,ID_ADHEtjTi,abambo odzikhweza akuchuluka kafukufuku wa a...
1,ID_AHfJktdQ,ambuye ziyaye ayamikira aphunzitsi tilitonse a...
2,ID_AUJIHpZr,anatcheleza akundiopseza gogo akundiopseza...
3,ID_AUKYBbIM,ulova wafika posauzana adatenga digiri ya up...
4,ID_AZnsVPEi,dzombe kukoma kuyambira makedzana pantha...
...,...,...
615,ID_zdpOUWyJ,kanyongolo wapempha oyimira milandu atsate mal...
616,ID_zhnOomuu,amandimenya zikomo gogo mtsikana wa zaka ...
617,ID_zmWHvBJb,apolisi athotha gulu la myp asilikali gu...
618,ID_zphjdFIb,mwambo wa ukwati wa chitonga mtundu wina uli...


# Rocchio (Website Score:- 0.6387) 
0.6419

In [14]:
from sklearn.neighbors import NearestCentroid
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
X_train = train_df_2.Text
X_test = test_df_2.Text
y_train = train_df_2.Label
print(X_train.shape,X_test.shape,y_train.shape)

(1436,) (620,) (1436,)


In [16]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', NearestCentroid()),
                     ])
text_clf.fit(X_train, y_train)
#predicted = text_clf.predict(X_test)
#print(metrics.classification_report(y_test, predicted))
#print(predicted)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', NearestCentroid())])

In [17]:
sample_df = pd.read_csv("/content/drive/MyDrive/CS 505 Project/SampleSubmission.csv")

In [18]:
sample_df

,ID,Label
0,ID_sQaPRMWO,0
1,ID_TanclvfR,0
2,ID_CNbveyvk,0
3,ID_MclKMhyP,0
4,ID_rNrmXOGD,0
...,...,...
615,ID_AHfJktdQ,0
616,ID_AUJIHpZr,0
617,ID_JhRAELrS,0
618,ID_QWVGFbvJ,0


In [19]:
submission_dict = {"ID":[],"Label":[]}

In [20]:
for i in sample_df.ID:
  submission_dict["ID"].append(i) #(test_df_2.loc[i,"ID"])
  text = test_df_2.loc[test_df_2['ID'] == i,"Text"].values[0]
  #print(text)
  input = [text]
  pred = text_clf.predict(input)
  submission_dict["Label"].append(pred.item())
submission_df = pd.DataFrame(submission_dict)

In [21]:
submission_df

,ID,Label
0,ID_sQaPRMWO,LAW/ORDER
1,ID_TanclvfR,SOCIAL
2,ID_CNbveyvk,SOCIAL ISSUES
3,ID_MclKMhyP,SOCIAL ISSUES
4,ID_rNrmXOGD,ECONOMY
...,...,...
615,ID_AHfJktdQ,RELIGION
616,ID_AUJIHpZr,RELATIONSHIPS
617,ID_JhRAELrS,POLITICS
618,ID_QWVGFbvJ,EDUCATION


In [22]:
submission_df.to_csv("/content/drive/MyDrive/CS 505 Project/Rocchio_Predicted_Submission.csv",index=False)

# Boosting (Website Score:- 0.5645) 

In [23]:
from sklearn.ensemble import GradientBoostingClassifier

In [24]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', GradientBoostingClassifier(n_estimators=100)),
                     ])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', GradientBoostingClassifier())])

In [25]:
submission_dict = {"ID":[],"Label":[]}

In [26]:
for i in sample_df.ID:
  submission_dict["ID"].append(i) #(test_df_2.loc[i,"ID"])
  text = test_df_2.loc[test_df_2['ID'] == i,"Text"].values[0]
  #print(text)
  input = [text]
  pred = text_clf.predict(input)
  submission_dict["Label"].append(pred.item())
submission_df = pd.DataFrame(submission_dict)

In [27]:
submission_df

,ID,Label
0,ID_sQaPRMWO,LAW/ORDER
1,ID_TanclvfR,SOCIAL
2,ID_CNbveyvk,POLITICS
3,ID_MclKMhyP,SOCIAL ISSUES
4,ID_rNrmXOGD,CULTURE
...,...,...
615,ID_AHfJktdQ,RELIGION
616,ID_AUJIHpZr,SOCIAL ISSUES
617,ID_JhRAELrS,POLITICS
618,ID_QWVGFbvJ,EDUCATION


In [28]:
submission_df.to_csv("/content/drive/MyDrive/CS 505 Project/Boosting_Predicted_Submission.csv",index=False)

# Bagging (Website Score:- 0.5871)

In [29]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [30]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', BaggingClassifier(KNeighborsClassifier())),
                     ])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 BaggingClassifier(base_estimator=KNeighborsClassifier()))])

In [31]:
submission_dict = {"ID":[],"Label":[]}

In [32]:
for i in sample_df.ID:
  submission_dict["ID"].append(i) #(test_df_2.loc[i,"ID"])
  text = test_df_2.loc[test_df_2['ID'] == i,"Text"].values[0]
  #print(text)
  input = [text]
  pred = text_clf.predict(input)
  submission_dict["Label"].append(pred.item())
submission_df = pd.DataFrame(submission_dict)

In [33]:
submission_df

,ID,Label
0,ID_sQaPRMWO,LAW/ORDER
1,ID_TanclvfR,RELIGION
2,ID_CNbveyvk,POLITICS
3,ID_MclKMhyP,SOCIAL ISSUES
4,ID_rNrmXOGD,ECONOMY
...,...,...
615,ID_AHfJktdQ,RELIGION
616,ID_AUJIHpZr,RELATIONSHIPS
617,ID_JhRAELrS,POLITICS
618,ID_QWVGFbvJ,EDUCATION


In [34]:
submission_df.to_csv("/content/drive/MyDrive/CS 505 Project/Bagging_Predicted_Submission.csv",index=False)